# Load Data

In [1]:
from pyearth import Earth
from DLtools.Data_preprocess import series_to_supervised,load_data
from DLtools.Data import load_data,instant_data,check_specific_col

import numpy as np
import pandas as pd

In [4]:
# loaddata = load_data()
# df_d = loaddata.daily()
# df_h = loaddata.hourly()
###################################
loading = instant_data()
df_h = loading.hourly_instant()
df_d = loading.daily_instant()


# Missing data handle

In [5]:
# data = df_h['2013-01-01':'2017-12-31']
data = df_d["2014-02-01":"2018-03-31"]

# limit 80% avaliable data only
for col in data.columns:
    if data[col].count()<len(data[col])*.8:
        data = data.drop(col, axis=1)
# Fill remain with mean
data = data.apply(lambda x: x.fillna(x.mean()),axis=0)
data.head()

,BKK001_rain,BKK002_rain,BKK005_rain,BKK007_rain,BKK008_rain,BKK013_rain,BKK018_rain,BKK019_rain,BKK020_rain,BKK021_rain,...,Dam_BH_Input_WaterVol,Dam_KheawNoi_Input_WaterVol,Dam_KiewKorMa_Input_WaterVol,Dam_KiewLom_Input_WaterVol,Dam_SK_Input_WaterVol,Dam_BH_Output_WaterVol,Dam_KheawNoi_Output_WaterVol,Dam_KiewKorMa_Output_WaterVol,Dam_KiewLom_Output_WaterVol,Dam_SK_Output_WaterVol
date,,,,,,,,,,,,,,,,,,,,,
2014-02-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.26,0.51,0.00,1.65,3.30,20.0,3.46,1.33,0.69,20.17
2014-02-02,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.0,...,0.00,0.79,0.16,1.73,3.42,20.0,3.46,1.59,0.69,20.02
2014-02-03,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.2,0.0,0.0,...,2.97,0.51,0.00,2.26,5.39,20.0,3.46,1.84,0.70,20.02
2014-02-04,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.2,0.0,0.0,...,0.00,0.23,0.00,2.53,1.64,20.0,3.46,2.27,0.70,20.12
2014-02-05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,4.78,0.20,0.49,2.31,3.54,20.0,3.46,3.10,0.71,20.05


In [6]:
TARGET = 'CPY012_wl'
## Shift target station to future 
n_out = 7                           #next 7 day

def to_supervise(data,target,n_out):
    data[target]=data[target].shift(-n_out)
    data = data.astype('float64').dropna()
    X = data.drop([target],axis=1)
    xlabels = list(X.columns)

    X = X.values
    y = data[target].values
    return X,y,xlabels

In [7]:
X,y,xlabels = to_supervise(data,TARGET,n_out)

# MARS feature selection (from all stations)

In [5]:
# #scale data since it's cannot run MARS in general
# from sklearn.preprocessing import MinMaxScaler
# # SCALE
# scaler = MinMaxScaler()
# data[data.columns] = scaler.fit_transform(data[data.columns])


In [8]:
#Fit an Earth model
criteria = ('rss', 'gcv', 'nb_subsets')
model = Earth(enable_pruning = True,
            #   max_degree=3,
            #  max_terms=20,
              minspan_alpha=.5,
              feature_importance_type=criteria,
              verbose=True)
model.fit(X,y,xlabels=xlabels)


Beginning forward pass
---------------------------------------------------------------
iter  parent  var  knot  mse       terms  gcv    rsq    grsq   
---------------------------------------------------------------
0     -       -    -     0.450707  1      0.451  0.000  0.000  
1     0       103  1286  0.067950  3      0.068  0.849  0.848  
2     0       225  963   0.059857  5      0.061  0.867  0.865  
3     0       117  1333  0.053503  7      0.055  0.881  0.879  
4     0       130  1288  0.049055  9      0.050  0.891  0.888  
5     0       136  72    0.044464  11     0.046  0.901  0.898  
6     0       134  892   0.042136  13     0.044  0.907  0.903  
7     0       228  725   0.040406  15     0.042  0.910  0.906  
8     0       105  1212  0.038134  17     0.040  0.915  0.911  
9     0       232  1346  0.036226  19     0.039  0.920  0.915  
10    0       102  623   0.034577  21     0.037  0.923  0.918  
11    0       217  446   0.032830  23     0.035  0.927  0.922  
12    0       107

Earth(feature_importance_type=('rss', 'gcv', 'nb_subsets'), minspan_alpha=0.5,
      verbose=True)

In [7]:
# #Print the model
# print(model.trace())
# print(model.summary())

* Feature Importance

In [9]:
print(model.summary_feature_importances(sort_by='rss')[:500])

                                   rss    gcv    nb_subsets
CPY006_wl                          0.79   0.80   0.04          
PAS008_wl                          0.08   0.08   0.04          
YOM009_wl                          0.04   0.04   0.08          
NAN006_wl                          0.02   0.02   0.06          
Dam_SK_Useable_WaterVol            0.02   0.02   0.04          
PAS002_wl                          0.01   0.01   0.04          
DIV006_wl                          0.01   0.01   0.04   


In [10]:
def toDF(rank):
    name,rss,gcv,nb_sub= list(),list(),list(),list()
    for i in range(len(rank)):
        if i%4==0:
            name.append(rank[i])
        elif i%4==1:
            rss.append(rank[i])
        elif i%4==2:
            gcv.append(rank[i])
        elif i%4==3:
            nb_sub.append(rank[i])
    data = {'feature':name,
    'rss':rss,
    'gcv':gcv,
    'nb_subset':nb_sub}
    score = pd.DataFrame(data)
    return score

In [11]:
nbsub = model.summary_feature_importances(sort_by='nb_subsets')[:2000].split()[3:83]
gcv = model.summary_feature_importances(sort_by='gcv')[:2000].split()[3:83]
rss = model.summary_feature_importances(sort_by='rss')[:2000].split()[3:83]


In [12]:
rss = toDF(rss)
gcv = toDF(gcv)
nbsub = toDF(nbsub)

In [13]:
#From top 20 rss,gcv,nbsub
top20 = pd.concat([rss,gcv,nbsub],ignore_index=True).drop_duplicates('feature')
top20

,feature,rss,gcv,nb_subset
0,CPY006_wl,0.79,0.80,0.04
1,PAS008_wl,0.08,0.08,0.04
2,YOM009_wl,0.04,0.04,0.08
3,NAN006_wl,0.02,0.02,0.06
4,Dam_SK_Useable_WaterVol,0.02,0.02,0.04
5,PAS002_wl,0.01,0.01,0.04
6,DIV006_wl,0.01,0.01,0.04
7,DNP032_humid,0.01,0.00,0.04
8,CPY013_wl,0.00,0.00,0.04
9,DIV002_wl,0.00,0.00,0.04


In [14]:
top20.to_csv('/home/song/Public/Song/Work/Thesis/MAR/[CPY012]_featurelist_MAR_daily_7d.csv',index=False)

# LOOP

In [17]:
data = df_h["2014-02-01":"2018-03-31"]
TARGET = 'CPY012_wl'
data = data.apply(lambda x: x.fillna(x.mean()),axis=0)
conclude_df=pd.DataFrame()

for n_out in range(1,24):
    X,y,xlabels = to_supervise(data,TARGET,n_out)
    criteria = ('rss', 'gcv', 'nb_subsets')
    model = Earth(enable_pruning = True,to_supervise
                #   max_degree=3,
                #  max_terms=20,
                minspan_alpha=.5,
                feature_importance_type=criteria,
                verbose=True)
    model.fit(X,y,xlabels=xlabels)
    nbsub = model.summary_feature_importances(sort_by='nb_subsets')[:2000].split()[3:83]
    gcv = model.summary_feature_importances(sort_by='gcv')[:2000].split()[3:83]
    rss = model.summary_feature_importances(sort_by='rss')[:2000].split()[3:83]
    
    rss,gcv,nbsub = toDF(rss),toDF(gcv),toDF(nbsub)
    top20=pd.concat([rss,gcv,nbsub],ignore_index=True)
    top20 = pd.concat([rss,gcv,nbsub],ignore_index=True).drop_duplicates('feature')
    top20['timestep'] = n_out

    #ADDED combine all result
    conclude_df = pd.concat([conclude_df,top20],ignore_index=True).drop_duplicates('feature')
    conclude_df.to_csv('/home/song/Public/Song/Work/Thesis/MAR/[CPY012]featurelist_MAR_hourly.csv',index=False)

0.794  
25    15  30     0.11  0.107  0.794  0.793  
26    2   29     0.11  0.108  0.793  0.792  
27    54  28     0.11  0.108  0.792  0.791  
28    31  27     0.11  0.109  0.791  0.790  
29    16  26     0.11  0.110  0.789  0.788  
30    21  25     0.11  0.111  0.787  0.787  
31    45  24     0.11  0.111  0.786  0.785  
32    43  23     0.11  0.112  0.784  0.783  
33    51  22     0.11  0.113  0.782  0.782  
34    50  21     0.11  0.114  0.781  0.780  
35    53  20     0.11  0.115  0.779  0.779  
36    9   19     0.12  0.115  0.778  0.777  
37    32  18     0.12  0.117  0.775  0.775  
38    37  17     0.12  0.118  0.772  0.772  
39    41  16     0.12  0.120  0.770  0.769  
40    48  15     0.12  0.121  0.767  0.766  
41    24  14     0.12  0.122  0.765  0.764  
42    7   13     0.12  0.124  0.761  0.761  
43    5   12     0.13  0.125  0.759  0.758  
44    6   11     0.13  0.128  0.754  0.754  
45    30  10     0.13  0.131  0.748  0.748  
46    38  9      0.13  0.134  0.742  0.742  
47